In [1]:
import glob
import os
import sys
import time
import random

import carla

import torch

import carla
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from PIL import Image

from agents.navigation.basic_agent import BasicAgent

import gym
import gym_carla

import torch.nn as nn
from itertools import count
import torch.nn.functional as F

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# parameters for the gym_carla environment
params = {
    'number_of_vehicles': 0,
    'number_of_walkers': 0,
    'display_size': 256,  # screen size of bird-eye render
    'max_past_step': 1,  # the number of past steps to draw
    'dt': 0.1,  # time interval between two frames
    'discrete': True,  # whether to use discrete control space
    'discrete_acc': [2.0],  # discrete value of accelerations
    'discrete_steer': [-0.3, 0.0, 0.3],  # discrete value of steering angles
    'continuous_accel_range': [-3.0, 3.0],  # continuous acceleration range
    'continuous_steer_range': [-0.3, 0.3],  # continuous steering angle range
    'ego_vehicle_filter': 'vehicle.lincoln*',  # filter for defining ego vehicle
    'port': 2000,  # connection port
    'town': 'Town04',  # which town to simulate
    'task_mode': 'random',  # mode of the task, [random, roundabout (only for Town03)]
    'max_time_episode': 500,  # maximum timesteps per episode
    'max_waypt': 12,  # maximum number of waypoints
    'obs_range': 32,  # observation range (meter)
    'lidar_bin': 0.125,  # bin size of lidar sensor (meter)
    'obs_size': 128, #obs image sizes
    'd_behind': 12,  # distance behind the ego vehicle (meter)
    'out_lane_thres': 1.8,  # threshold for out of lane
    'desired_speed': 5,  # desired speed (m/s)
    'max_ego_spawn_times': 20,  # maximum times to spawn ego vehicle
    'display_route': True,  # whether to render the desired route
    'pixor_size': 64,  # size of the pixor labels
    'pixor': False,  # whether to output PIXOR observation
    'use_fixed':False,
    'weather':'ClearNoon',
    'Collect_Data':True
}

# Set gym-carla environment
env = gym.make('carla-v0', params=params)

connecting to Carla server...
Carla server connected!
WeatherParameters(cloudiness=5.000000, cloudiness=5.000000, precipitation=0.000000, precipitation_deposits=0.000000, wind_intensity=10.000000, sun_azimuth_angle=-1.000000, sun_altitude_angle=45.000000, fog_density=2.000000, fog_distance=0.750000, fog_falloff=0.100000, wetness=0.000000, scattering_intensity=1.000000, mie_scattering_scale=0.030000, rayleigh_scattering_scale=0.033100)
372


In [5]:
class PerceptionNet(nn.Module):

    def __init__(self):
        super(PerceptionNet,self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        self.conv5 = nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        
        self.conv6 = nn.Conv2d(512, 64, kernel_size=4, stride=1)
        
        self.conv7 = torch.nn.ConvTranspose2d(64,512, kernel_size =4, stride=1)
        self.bn6 = nn.BatchNorm2d(512)
        
        self.conv8 = torch.nn.ConvTranspose2d(512,256, kernel_size =4, stride=2, padding=1)
        self.bn7 = nn.BatchNorm2d(256)
        
        self.conv9 = torch.nn.ConvTranspose2d(256,128, kernel_size =4, stride=2, padding=1)
        self.bn8 = nn.BatchNorm2d(128)
        
        self.conv10 = torch.nn.ConvTranspose2d(128,64, kernel_size =4, stride=2, padding=1)
        self.bn9 = nn.BatchNorm2d(64)
        
        self.conv11 = torch.nn.ConvTranspose2d(64,32, kernel_size =4, stride=2, padding=1)
        self.bn10 = nn.BatchNorm2d(32)
        
        self.conv12 = torch.nn.ConvTranspose2d(32,23, kernel_size =4, stride=2,padding=1)
        
            
    def encode(self, x):
        x = F.leaky_relu(self.conv1(x),negative_slope=0.02)
        x = F.leaky_relu(self.bn2(self.conv2(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn3(self.conv3(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn4(self.conv4(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn5(self.conv5(x)),negative_slope=0.02)
        return self.conv6(x)
    
    def decode(self, x):
        x = F.leaky_relu(self.bn6(self.conv7(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn7(self.conv8(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn8(self.conv9(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn9(self.conv10(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn10(self.conv11(x)),negative_slope=0.02)
        return F.log_softmax(self.conv12(x),dim=1)
    
    def forward(self, x):
        x = x.to(device, dtype=torch.float32)
        latent = self.encode(x)
        out = self.decode(latent)
        return out, latent

In [8]:
class DQN(nn.Module):

    def __init__(self, outputs):
        super(DQN, self).__init__()
        
        self.lin1 = nn.Linear(9,80)
        self.lin2 = nn.Linear(80,50)
        self.lin3 = nn.Linear(50,25)
        self.lin4 = nn.Linear(25,15)
        self.lin5 = nn.Linear(15,8)
        self.lin6 = nn.Linear(8,3)

    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        x = F.relu(self.lin4(x))
        x = F.relu(self.lin5(x))
        x = self.lin6(x)
        
        return x.view(x.size(0), -1)

In [9]:
model = PerceptionNet()
model.to(device)
model.load_state_dict(torch.load('./AE_params/model_3.best'))
model.eval()

vis_net = DQN(3).to(device)
vis_net.load_state_dict(torch.load('./model_params_CL/model_10.final'))
vis_net.eval()

DQN(
  (lin1): Linear(in_features=9, out_features=80, bias=True)
  (lin2): Linear(in_features=80, out_features=50, bias=True)
  (lin3): Linear(in_features=50, out_features=25, bias=True)
  (lin4): Linear(in_features=25, out_features=15, bias=True)
  (lin5): Linear(in_features=15, out_features=8, bias=True)
  (lin6): Linear(in_features=8, out_features=3, bias=True)
)

In [8]:
num_episodes = 500
min_overall_loss = 1000
data = DataCollector(env,test=False)
for i_episode in range(num_episodes):
    rewards = 0
    # Initialize the environment and state
    obs = env.reset()
    #ego_dir retirves the distance and angle from vehicle to nearest waypoint
    ego_location = env.ego.get_location()
    ego_dir = gym_carla.envs.misc.get_lane_dis(env.waypoints,ego_location.x,ego_location.y)
    #pos gets a distanc d and array w which has to be seperated out in below line
    ego_pos = np.asarray((ego_dir[0],ego_dir[1][0],ego_dir[1][1]),dtype=np.float32)
    state = np.concatenate((ego_pos,np.zeros(6)))
    
    #inital state with previous action for data collection consistency
    action = np.array([0])
    reward = 0
    
    data.collect_step(obs, state, reward, action)
    
    state = torch.tensor(state)
    loss = episode_loss = 1000
    for t in count():
        # Select and perform an action
        with torch.no_grad():
            action = vis_net(state.float()).argmax().view(1,1)
        next_obs, reward, done, info  = env.step(action.item())
        rewards += reward
    
        #pos gets a distanc d and array w which has to be seperated out in below line
        pos = np.asarray((info['position'][0],info['position'][1][0],info['position'][1][1]))
        ang = np.asarray(info['angular_vel'])
        acc = np.asarray(info['acceleration'])
        steer = np.asarray(info['steer'])
        next_state = np.concatenate((pos, ang, acc, steer), axis=None)
        
        #collect data
        
        data.collect_step(obs,state,reward,np.array([action.item()]))
        
        #update state
        state = torch.tensor(next_state)
        obs = next_obs
        #obs = torch.tensor(next_obs)
        
        
        if done:
            data.stop_recording()
            break
print('Done')

Done


In [6]:
import time
import os
class DataCollector():
    def __init__(self, env, recorder = False, test=False):
        self.timestamp = time.strftime("%a, %d %b %Y %H:%M:%S")
        if not env:
            raise Exception("Please provide a carla gym env object")
        self.recorder = recorder
        
        dir_str = './Datasets/'+env.weather+'/'+env.town
        
        #collect test set
        if test:
            dir_str = dir_str +'/test'
            
        self.rgb_dir = dir_str+'/RGB/'
        self.sem_dir = dir_str+'/Semantic/'
        self.state_dir = dir_str+'/States/'
        self.recording_dir = dir_str+'/Recordings/'
        if not os.path.isdir(dir_str):
            os.makedirs(dir_str)
            os.makedirs(self.rgb_dir)
            os.makedirs(self.sem_dir)
            os.makedirs(self.state_dir)
            os.makedirs(self.recording_dir)
        
        if self.recorder:
            self.record()
    
            
    
    def collect_step(self, obs, state, reward, action):
        self.timestamp = time.strftime("%a, %d %b %Y %H:%M:%S")
        # save image data 
        im = Image.fromarray(obs['camera'])
        im.save(self.rgb_dir+self.timestamp+".jpeg")
        np.save(self.sem_dir+self.timestamp,obs['semantic'][:, :, :1])
        #convert tensor to numpy array and append action to state vector
        if torch.is_tensor(state):
            state = state.cpu()
            state = state.numpy()
        state = np.append(state,action)
        np.save(self.state_dir+self.timestamp,state)
        
        
        
        
    def record(self):
        env.client.start_recorder(self.recording_dir+self.timestamp+".log")

    def stop_recording(self):
        if self.recorder:
            env.client.stop_recorder()


In [22]:
obs['camera'].dtype

dtype('uint8')

In [1]:
Image.fromarray(obs['camera'],'RGB')

NameError: name 'Image' is not defined

In [2]:
import matplotlib.pyplot as plt